# Measuring Variability
This csv file below contains rough estimates of the baseline values for these metrics (again, these numbers have been changed from Udacity's true numbers).

For each of the metrics the standard deviation is calculated for a sample size of 5000 unique cookies visiting the course overview page.

## baseline values

In [2]:
#import packages
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

#import csv file
baseline = pd.read_excel('C:\Users\Silvia\Documents\GitHub\Projects-portfolio\ABTestingUdacity\Final Project Baseline Values.xlsx', header=None)
baseline

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2544766989.py, line 7)

In [ ]:
# adjust the dataset 
baseline.columns = ['Metrics','Estimator']
baseline['Metrics'] = ['cookies', 'cliks', 'user-id','click-through-probability','gross conversion','retention','net conversion']
baseline['dmin'] = [3000, 240, -50, 0.01, -0.01, 0.01, 0.0075]
baseline